In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.stats import spearmanr
import seaborn as sns
import re
import itertools
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline 
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.pipeline import Pipeline
from matplotlib.colors import LinearSegmentedColormap
from plot import plot_confusion
from plot import plot_cmap
import numpy as np
mpl.rcParams['font.sans-serif'] = ['Arial']
mpl.rcParams['xtick.labelsize'] = 18
mpl.rcParams['ytick.labelsize'] = 18
mpl.rcParams.update({'font.size': 20})
mpl.rcParams['font.sans-serif'] = ['Arial']
mpl.rcParams['xtick.labelsize'] = 18
mpl.rcParams['ytick.labelsize'] = 18
mpl.rcParams.update({'font.size': 24})

color = ['tab:blue', 'tab:green', 'tab:orange', 'tab:red', 'tab:olive', 'tab:purple',
         'tab:cyan', 'tab:gray', 'tab:brown', 'black', 'darkgreen', 'deeppink']
grid_color = ['tab:blue', 'tab:green', 'tab:orange', 'tab:red', 'tab:olive', 'tab:purple',
         'tab:cyan', 'tab:gray', 'tab:brown', 'black', 'darkgreen', 'deeppink']


In [ ]:
class Main:
    def __init__(self, input_file='mini project_v5.xlsx'):
        self.input_file = input_file
        self.formula = None
        self.infor = None
    def rand_jitter(arr):
        np.random.seed(7)
        if len(arr) == 0:
            stdev = 0
        else:
            stdev = 0.04 * (max(arr) - min(arr))
        jitter = arr + (np.random.randn(len(arr)) * stdev)
        return jitter

    def cal_rsq(x,y):
        rsq = spearmanr(x, y)[0]
        return rsq

    def process(self):
        self.formula = pd.read_excel(self.input_file, sheet_name='formula')
        self.infor = pd.read_excel(self.input_file, sheet_name='infor')
        self.formula.columns = self.formula.columns.str.strip()

        mw_dict = dict(zip(self.infor['compound'], self.infor['MW']))
        vabc_dict = dict(zip(self.infor['compound'], self.infor['Vabc']))
        o_dict = dict(zip(self.infor['compound'], self.infor['O']))
        f_dict = dict(zip(self.infor['compound'], self.infor['F']))
        c_dict = dict(zip(self.infor['compound'], self.infor['C']))
        h_dict = dict(zip(self.infor['compound'], self.infor['H']))
        SLogP_dict = dict(zip(self.infor['compound'], self.infor['SLogP']))
        cyclic_dict = dict(zip(self.infor['compound'], self.infor['cyclic']))

        # Process formula data
        for index, row in formula.iterrows():
            formula_str = str(row['formula']).strip()
            match = re.match(r"([A-Za-z0-9\-]+(?:-[A-Za-z0-9\-]+)*)\s*\(([\d\.\-]+(?:-[\d\.\-]+)*)\s*mol\)", formula_str)
            if not match:
                continue
                
            compounds_part, mols_part = match.groups()
            compounds = compounds_part.split("-")
            compounds = [compound + " " for compound in compounds]
            original_mols = [float(m) for m in mols_part.split("-")]
                
            if len(compounds) != len(original_mols):
                continue
                    
            # Tính tổng khối lượng của các hợp chất
            total_mass = 0
            for compound, mol in zip(compounds, original_mols):
                for k in mw_dict.keys():
                    if compound in k:
                        total_mass += mw_dict[k] * mol if pd.notna(mw_dict[k]) else 0
                        break
            scale_factor = 100 / total_mass if total_mass > 0 else 0
            for i, (compound, original_mols) in enumerate(zip(compounds, original_mols), start=1):
                formula.loc[index, f"c{i}"] = compound
                formula.loc[index, f"n{i}"] = original_mols * scale_factor
        # tính số mol của Li và sol
        for i in formula.index:
            s_count = 1  
            a_count = 1
            Li = 0
            sol = 0
            
            for j in range(1, 9):
                c_col = f'c{j}'
                n_col = f'n{j}'
                if c_col in formula.columns and pd.notna(formula.loc[i, c_col]):
                    compound = formula.loc[i, c_col]
                    n_value = formula.loc[i, n_col]
                    
                    if 'Li' in compound:
                        Li += n_value
                        for k in o_dict.keys():
                            if compound in k:
                                formula.loc[i, f'a{a_count}O'] = o_dict[k] if pd.notna(o_dict[k]) else 0
                                formula.loc[i, f'a{a_count}F'] = f_dict[k] if pd.notna(f_dict[k]) else 0
                                a_count += 1
                                break
                    else:
                        sol += n_value
                        for k in o_dict.keys():
                            if compound in k:
                                formula.loc[i, f's{s_count}C'] = c_dict[k] if pd.notna(c_dict[k]) else 0
                                formula.loc[i, f's{s_count}H'] = h_dict[k] if pd.notna(h_dict[k]) else 0
                                formula.loc[i, f's{s_count}O'] = o_dict[k] if pd.notna(o_dict[k]) else 0
                                formula.loc[i, f's{s_count}F'] = f_dict[k] if pd.notna(f_dict[k]) else 0
                                s_count += 1
                                break
            
            formula.loc[i, 'Li'] = Li
            formula.loc[i, 'sol'] = sol
            formula.loc[i, 'Li/sol'] = round(Li/sol, 3) if sol != 0 else 0
        # tính tỉ lệ số mol của các nguyên tố
        prefix_suffix_pairs = [
            ('a', 'O'),
            ('a', 'F'),
            ('s', 'C'),
            ('s', 'H'),
            ('s', 'O'),
            ('s', 'F'),
            ('s', 'P'),
            ('s', 'S')
        ]

        for prefix, suffix in prefix_suffix_pairs:
            cols = [col for col in formula.columns if col.startswith(prefix) and col.endswith(suffix) and len(col) == 3]
            formula[prefix + suffix] = formula[cols].sum(axis=1, skipna=True)

        formula['sF/sC'] = formula.apply(lambda x: round(x['sF'] / x['sC'], 2) if pd.notna(x['sF']) and pd.notna(x['sC']) and x['sC'] != 0 else 0, axis=1)
        formula['sF/sO'] = formula.apply(lambda x: round(x['sF'] / x['sO'], 2) if pd.notna(x['sF']) and pd.notna(x['sO']) and x['sO'] != 0 else 0, axis=1)
        formula['sC.sH'] = formula.apply(lambda x: round(x['sC'] * x['sH'], 2) if pd.notna(x['sC']) and pd.notna(x['sH']) and x['sH'] != 0 else 0, axis=1)
        formula['sO/sC'] = formula.apply(lambda x: round(x['sO'] / x['sC'], 2) if pd.notna(x['sO']) and pd.notna(x['sC']) and x['sC'] != 0 else 0, axis=1)
        # tính số nối đôi O trong các hợp chất
        smiles_dict = dict(zip(infor['compound'], infor['SMILES']))
        formula['dbO'] = 0
        for i in formula.index:
            db_count = 0
            for j in range(1, 9):
                c_col = f'c{j}'
                if c_col in formula.columns and pd.notna(formula.loc[i, c_col]):
                    compound = formula.loc[i, c_col]
                    for k in smiles_dict.keys():
                        if compound in k and pd.notna(smiles_dict[k]):
                            db_count += smiles_dict[k].count('=O')
                            break
            formula.loc[i, 'dbO'] = db_count
        formula['sbO'] = formula['sO'] - formula['dbO']
        # tính pct
        for i in range(1, 9):
            c_col = f'c{i}'
            n_col = f'n{i}'
            
            if c_col in formula.columns:
                pct_col = f'%n{i}'
                formula[pct_col] = formula.apply(
                    lambda row: round(row[n_col] / row['sol'] * 100, 2) if pd.notna(row[c_col]) and 'Li' not in str(row[c_col]) else np.nan,
                    axis=1
                )
        # tính density
        for i in formula.index:
            total_mw = 0
            total_vabc = 0
            for j in range(1, 9):
                c_col = f'c{j}'
                n_col = f'n{j}'
                if c_col in formula.columns and pd.notna(formula.loc[i, c_col]):
                    compound = formula.loc[i, c_col]
                    n_value = formula.loc[i, n_col]
                    if 'Li' not in compound:
                        for k in mw_dict.keys():
                            if compound in k:
                                total_mw += mw_dict[k] * n_value if pd.notna(mw_dict[k]) else 0
                                total_vabc += vabc_dict[k] * n_value if pd.notna(vabc_dict[k]) else 0
                                break
            
            formula.loc[i, 'density'] = round(total_mw/total_vabc, 3) if total_vabc != 0 else 0
        # tính radius, apol, bpol, Vabc
        properties = ['Radius', 'Vabc', 'apol', 'bpol']
        property_dicts = {prop: dict(zip(infor['compound'], infor[prop])) for prop in properties}

        for prop in properties:
            formula[f'{prop}'] = formula[prop].astype(float)  
            formula[f'{prop}-'] = formula[f'{prop}-'].astype(float)
            formula[f'{prop}-r'] = formula[f'{prop}-r'].astype(float)
            formula[f'{prop}+'] = formula[f'{prop}+'].astype(float)
            formula[prop] = 0
            formula[f'{prop}-'] = 1
        for i in formula.index:
            for prop in properties:
                total_value = 0
                minus_value = 1
                for j in range(1, 9):
                    c_col = f'c{j}'
                    n_col = f'n{j}'
                    pct_col = f'%n{j}'
                    if c_col in formula.columns and pd.notna(formula.loc[i, c_col]):
                        compound = formula.loc[i, c_col]
                        n_value = formula.loc[i, n_col]
                        n2_value = formula.loc[i, pct_col]
                        if 'Li' not in compound:
                            for k in property_dicts[prop].keys():
                                if compound in k:
                                    total_value += property_dicts[prop][k] * n_value if pd.notna(property_dicts[prop][k]) else 0
                                    minus_value *= property_dicts[prop][k] ** n2_value if pd.notna(property_dicts[prop][k]) else 1
                                    break
                formula.loc[i, prop] = round(total_value, 3) if total_value != 0 else 0
                formula[f'{prop}-'] = formula[f'{prop}-'].astype(float)
                formula.loc[i, f'{prop}-'] = round(minus_value, 3) if minus_value != 1 else 1
        for prop in properties:
            formula[f'{prop}-r'] = formula.apply(lambda x: round(x[prop] / x['sol'], 3) if x['sol'] != 0 else 0, axis=1)
            formula[f'{prop}+'] = formula.apply(lambda x: round((2 * x[f'{prop}-r']) - x[f'{prop}-'], 3), axis=1)
        # tính SLogP
        for i in formula.index:
            total_slogp = 0
            for j in range(1, 9):
                c_col = f'c{j}'
                pct_col = f'%n{j}'
                if c_col in formula.columns and pd.notna(formula.loc[i, c_col]):
                    compound = formula.loc[i, c_col]
                    n2_value = formula.loc[i, pct_col]
                    if 'Li' not in compound:
                        for k in SLogP_dict.keys():
                            if compound in k:
                                if pd.notna(SLogP_dict[k]) and pd.notna(n2_value):
                                    total_slogp += 10**SLogP_dict[k] * n2_value
                                break
                                
            formula.loc[i, 'SLogP-r'] = round(np.log10(total_slogp), 3) if total_slogp > 0 else 0
        # tính cyclic
        for i in formula.index:
            total_cyclic = 0
            for j in range(1, 9):
                c_col = f'c{j}'
                n_col = f'n{j}'
                if c_col in formula.columns and pd.notna(formula.loc[i, c_col]):
                    compound = formula.loc[i, c_col]
                    n_value = formula.loc[i, n_col]
                    if 'Li' not in compound:
                        for k in cyclic_dict.keys():
                            if compound in k:
                                total_cyclic += cyclic_dict[k] if pd.notna(cyclic_dict[k]) else 0
                                break
            formula.loc[i, 'cyclic'] = round(total_cyclic, 3) if total_cyclic != 0 else 0
        # tính pi
        formula['pi'] = 0
        for i in formula.index:
            pi_count = 0
            for j in range(1, 9):
                c_col = f'c{j}'
                if c_col in formula.columns and pd.notna(formula.loc[i, c_col]):
                    compound = formula.loc[i, c_col]
                    if 'Li' not in compound:  # Only count for non-Li compounds
                        for k in smiles_dict.keys():
                            if compound in k and pd.notna(smiles_dict[k]):
                                pi_count += smiles_dict[k].count('=')  # Count all double bonds
                                break
            formula.loc[i, 'pi'] = pi_count

    def plot_results(self):
        # Implement visualization using matplotlib and seaborn
        pass

    def regression(self):
        features = ['Li', 'aF', 'aO', 'sC', 'sO', 'sF', 'pi', 'cyclic', 'apol', 'Vabc-', 'SLogP-r', 'Radius-r']
        y = ['LCEi', 'LCEs', 'LCEs']
        names = [cycle, cycle, aurbach]
        tags = ['a)', 'd)', 'b)', 'e)', 'c)', 'f)']
        split = 0.8
        random_state = 7
        #################################### Settings #################################### 
        fig, axs = plt.subplots(2,3, figsize=(28, 14), dpi = 192)
        scaler = MinMaxScaler()
        count=0

        model_cycle = RandomForestRegressor(n_estimators=140,
                                            max_depth=30,
                                            criterion='absolute_error',
                                            random_state=random_state)

        model_aurbach = RandomForestRegressor(n_estimators=340,
                                            max_depth=24,
                                            criterion='absolute_error',
                                            random_state=random_state)

        models = [model_cycle, model_cycle, model_aurbach]

        #################################### Running #################################### 
        for j in range(3):
            file = names[j]
            standardized_data = scaler.fit_transform(file[features])
            df_std = pd.DataFrame(standardized_data, columns=file[features].columns)
            X = df_std[features]
            Y = file[y[j]]

            x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size = split, random_state=random_state)

            # Train model
            model = models[j].fit(x_train, y_train)
            y_train_predict = model.predict(x_train)
            y_test_predict = model.predict(x_test)
            y_predict = model.predict(X)
            
            importances = model.feature_importances_
            sorted_indices = np.argsort(importances)[::-1]

            # Evaluate model:
            rmse = root_mean_squared_error(Y, y_predict)
            rsq = r2_score(Y, y_predict)
                
            # Create bar plot with viridis colormap
            colors = plt.cm.viridis(np.linspace(0, 1, len(importances[sorted_indices])))
            axs[0,j].bar(range(x_train.shape[1]), importances[sorted_indices], color=colors)

            axs[0,j].set_ylabel("Relative importance", fontsize=18)
            #axs[0,j].set_ylabel("Mean decrease in impurity", fontsize=18)
            axs[0,j].set_xticks(range(x_train.shape[1]), x_train.columns[sorted_indices], rotation=0, fontsize=14)
            axs[0,j].set_xticks(range(x_train.shape[1]), [features[i] for i in sorted_indices], rotation = 30)
            axs[0,j].text(0.9, 0.95, tags[count], fontsize=26, c='black', verticalalignment='center', weight='bold', transform=axs[0,j].transAxes)

            axs[1,j].scatter(y_train, y_train_predict, edgecolor='black', linewidth=1, s = 50, alpha = 0.6, c="#e84629", label='train')
            axs[1,j].scatter(y_test, y_test_predict, edgecolor='black', linewidth=1, s = 50, alpha = 0.7, c="darkcyan", label='test')
            axs[1,j].plot(Y, Y, c = 'black', linewidth = 2)
            
            axs[1,j].set_xlabel(f"Observed {y[j]}")
            axs[1,j].set_ylabel(f"Predicted {y[j]}")
            axs[1,j].legend(loc='lower right', )
            axs[1,j].grid(color='black', linestyle='--', linewidth=1, alpha=0.2)
            axs[1,j].spines["top"].set_visible(False)
            axs[1,j].spines["right"].set_visible(False)
            axs[1,j].spines["bottom"].set_visible(False)
            axs[1,j].spines["left"].set_visible(False)
            
            axs[1,j].text(0.9, 1, tags[count+1], fontsize=26, c='black', verticalalignment='center', weight='bold', transform=axs[1,j].transAxes)
            axs[1,j].text(0.05, 0.9, f'R$^2$: {round(rsq, 4)}', fontsize = 18, transform=axs[1,j].transAxes)
            axs[1,j].text(0.05, 0.83, 'RMSE: {:0.3f}'.format(rmse), fontsize = 18, transform=axs[1,j].transAxes)
            
            count+=2
            
        axs[0,0].text(0.45, 0.95, 'Cycle', fontsize=32, c='darkgoldenrod', verticalalignment='center', transform=axs[0,0].transAxes)
        axs[0,1].text(0.45, 0.95, 'Cycle', fontsize=32, c='darkgoldenrod', verticalalignment='center', transform=axs[0,1].transAxes)
        axs[0,2].text(0.45, 0.95, 'Aurbach', fontsize=32, c='teal', verticalalignment='center', transform=axs[0,2].transAxes)
        axs[1,2].set_xlabel(f"Observed LCE")
        axs[1,2].set_ylabel(f"Predicted LCE")

        #plt.savefig(f'Feature importance_{type(model_cycle).__name__}.tif', bbox_inches = 'tight')
        plt.show()

    def classification(self):
        features = ['Li', 'aF', 'aO', 'sC', 'sO', 'sF', 'pi', 'cyclic', 'apol', 'Vabc-', 'SLogP-r', 'Radius-r']
        y = 'class'
        names = [cycle, aurbach]
        colors = ['darkgoldenrod', 'teal']
        titles = ['Cycle', 'Aurbach']
        split = 0.8

        #################################### Settings ####################################
        fig, axs = plt.subplots(2,2,figsize=(10, 8), dpi = 192)
        scaler = MinMaxScaler()

        model_cycle = GradientBoostingClassifier(random_state=random_state,
                                                learning_rate=0.1,
                                                loss='exponential',
                                                criterion='friedman_mse',
                                                n_estimators=462,
                                                max_depth=25,
                                                min_samples_leaf=2,
                                                min_samples_split=2)

        model_aurbach = RandomForestClassifier(bootstrap=True,
                                            criterion='gini',
                                            max_features='sqrt',
                                            min_samples_split=2,
                                            min_samples_leaf=38,
                                            max_depth=1,
                                            n_estimators=28,
                                            random_state=random_state)

        #################################### Running ####################################
        models = [model_cycle, model_aurbach]

        for j in range(2):
            file = names[j]
            standardized_data = scaler.fit_transform(file[features])
            df_std = pd.DataFrame(standardized_data, columns=file[features].columns)
            X = df_std[features]
            Y = file[y]
            x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size=split, 
                                                                random_state=random_state,
                                                                stratify=Y)
            
            try:
                clf = models[j].fit(x_train, y_train)
                y_predict = clf.predict(X)
            except ValueError as e:
                print(f"Error in {titles[j]} model: {str(e)}")
                print(f"Class distribution in training set: {np.bincount(y_train)}")
                continue
            accuracy = accuracy_score(Y, y_predict)
            precision = precision_score(Y, y_predict)
            recall = recall_score(Y, y_predict)
            print('Accuracy: {:.03}% \n'.format(accuracy*100))
            
            # Confusion matrix:
            cnf_matrix = confusion_matrix(Y, y_predict)
            cnf_matrix_norm = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1, keepdims = True)
            plot_confusion(Y, matrix=cnf_matrix, ax=axs[j,0], title = 'Confusion matrix', shrink=1)
            plot_confusion(Y, matrix=cnf_matrix_norm, ax=axs[j,1], title = 'Normalized confusion matrix', rf = '.2f', shrink=1)

            # Add text annotations
            axs[j,0].text(1.65, 1.2, titles[j], fontsize=22, c=colors[j], verticalalignment='center', transform=axs[j,0].transAxes)
            axs[j,0].text(1.5, 0.7, 'Precision: {:.03}% \n'.format(precision*100), fontsize=15, c=colors[j], verticalalignment='center', transform=axs[j,0].transAxes)
            axs[j,0].text(1.5, 0.55, 'Recall: {:.03}% \n'.format(recall*100), fontsize=15, c=colors[j], verticalalignment='center', transform=axs[j,0].transAxes)

            #save model in pkl file:
            pkl_filename= f'{type(models[j]).__name__}_{titles[j]}.pkl'
            with open(pkl_filename, 'wb') as file:  
                pickle.dump(models[j], file)
            with open(pkl_filename, 'rb') as file:  
                saved_model = pickle.load(file)
            print(saved_model)

        fig.tight_layout()
        #plt.savefig(f'Classification_performance.tif', bbox_inches = 'tight')
        plt.show()

    def run(self):
        self.process()
        self.plot_results()
        self.regression()
        self.classification()

if __name__ == "__main__":
    main = Main()
    main.run()
